In [ ]:
# !lscpu | grep 'Model name'

In [ ]:
# !git clone https://github.com/datvodinh10/recurrent-ppo.git

In [ ]:
import torch
import torch.nn as nn

In [ ]:
%cd recurrent-ppo/ENV

In [ ]:
from model.trainer import *
import env


In [ ]:
game_name = "MachiKoro"
env.make(game_name)
writer_path = f"/content/runs/{game_name}/"

In [ ]:
env.getActionSize()

In [ ]:
max_eps_length = 200

In [ ]:
config = {
    "critic_coef": 1,
    "entropy_coef": 0,
    "num_epochs": 3,
    "max_eps_length":max_eps_length,
    "seq_length":8,
    "hidden_size":128,
    "lr": 2.5e-4,
    "num_game_per_batch":128,
    "gamma":0.99,
    "gae_lambda":0.95,
    "policy_kl_range":0.0008,
    "policy_params": 20,
    "value_clip": 1.0,
    "max_grad_norm": 0.5,
    "n_mini_batch": 4,
    "rewards": [-1,1,0], #[lose,win,not_done]
    "set_detect_anomaly": True,
    "normlize_advantage": True,

}


In [ ]:
trainer = Trainer(config=config,env=env,writer_path = writer_path)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/runs/

In [ ]:
trainer.train()